# Proyecto 2 - Deep Learning

## Integrantes

- Santiago Florez - 
- Javier Barrera - 202214779
- Ana

## Punto 1

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torchvision.models import ResNet18_Weights
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
# =====================================
# CONFIGURAR DISPOSITIVO
# =====================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

Usando dispositivo: cpu


In [3]:
# =====================================
# TRANSFORMACIONES (3 canales para ResNet)
# =====================================
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [7]:
# =====================================
# CARGA DE DATOS Y PARTICIÓN
# =====================================
dataset = datasets.ImageFolder("DataSnake/CLASIFICADOR SNAKES")
class_names = dataset.classes

targets = np.array(dataset.targets)

train_idx, test_idx = train_test_split(
    np.arange(len(targets)),
    test_size=0.1,
    stratify=targets,
    random_state=42
)

train_dataset = datasets.ImageFolder("DataSnake/CLASIFICADOR SNAKES", transform=transform_train)
test_dataset = datasets.ImageFolder("DataSnake/CLASIFICADOR SNAKES", transform=transform_test)

train_dataset = Subset(train_dataset, train_idx)
test_dataset = Subset(test_dataset, test_idx)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

In [8]:
# =====================================
# CARGAR RESNET18 PREENTRENADA Y MODIFICAR CABEZA
# =====================================
resnet18 = models.resnet18(weights=ResNet18_Weights.DEFAULT)

# Congelar capas convolucionales
for param in resnet18.parameters():
    param.requires_grad = False

num_features = resnet18.fc.in_features
resnet18.fc = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, len(class_names))
)

resnet18 = resnet18.to(device)

optimizer = optim.Adam(resnet18.fc.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\javier/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100.0%


In [9]:
# =====================================
# FUNCIONES DE ENTRENAMIENTO Y EVALUACIÓN
# =====================================
def evaluate_model(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total


def train_model(model, train_loader, test_loader, optimizer, criterion, epochs=10):
    history = {"train_loss": [], "test_acc": []}

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * inputs.size(0)

        avg_loss = total_loss / len(train_loader.dataset)
        acc = evaluate_model(model, test_loader)

        history["train_loss"].append(avg_loss)
        history["test_acc"].append(acc)

        print(f"Época {epoch}/{epochs} - Loss: {avg_loss:.4f} | Test Acc: {acc:.4f}")

    return history

In [ ]:
# =====================================
# ENTRENAMIENTO
# =====================================
print("\nEntrenando modelo ResNet18...")
history = train_model(resnet18, train_loader, test_loader, optimizer, criterion, epochs=30)

final_acc = evaluate_model(resnet18, test_loader)
print(f"\n✅ Accuracy final de ResNet18: {final_acc:.2f}")


Entrenando modelo ResNet18...


C:\Users\javier\miniconda3\envs\dl_20252\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Época 1/30 - Loss: 1.0950 | Test Acc: 0.4211
Época 2/30 - Loss: 0.9480 | Test Acc: 0.5789
Época 3/30 - Loss: 0.8216 | Test Acc: 0.8947
Época 4/30 - Loss: 0.7287 | Test Acc: 0.5263


In [ ]:
# =====================================
# PLOT DE LOSS Y ACCURACY
# =====================================
epochs = range(1, len(history["train_loss"]) + 1)
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.plot(epochs, history["train_loss"], 'b-o')
plt.title("Pérdida (Loss) de entrenamiento")
plt.xlabel("Épocas")
plt.ylabel("Loss")

plt.subplot(1,2,2)
plt.plot(epochs, history["test_acc"], 'r-o')
plt.title("Exactitud (Accuracy) en Test")
plt.xlabel("Épocas")
plt.ylabel("Accuracy")

plt.tight_layout()
plt.show()

## Punto 2 

In [ ]:
# =====================================
# TRANSFORMACIONES (3 canales para ResNet)
# =====================================
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [ ]:
# =====================================
# CARGA DE DATOS Y PARTICIÓN
# =====================================
dataset = datasets.ImageFolder("DataSnake/CLASIFICADOR SNAKES")
class_names = dataset.classes
targets = np.array(dataset.targets)

train_idx, test_idx = train_test_split(
    np.arange(len(targets)),
    test_size=0.1,
    stratify=targets,
    random_state=42
)

train_dataset = datasets.ImageFolder("DataSnake/CLASIFICADOR SNAKES", transform=transform_train)
test_dataset = datasets.ImageFolder("DataSnake/CLASIFICADOR SNAKES", transform=transform_test)

train_dataset = Subset(train_dataset, train_idx)
test_dataset = Subset(test_dataset, test_idx)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)


In [ ]:
# =====================================
# CARGAR RESNET18 PREENTRENADA (fine-tuning completo)
# =====================================
resnet18 = models.resnet18(weights=ResNet18_Weights.DEFAULT)

# 🔓 Descongelar TODAS las capas (fine-tuning completo)
for param in resnet18.parameters():
    param.requires_grad = True

# Reemplazar la capa final (fc)
num_features = resnet18.fc.in_features
resnet18.fc = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, len(class_names))
)

resnet18 = resnet18.to(device)

# 🔥 Importante: tasa de aprendizaje más baja al hacer fine-tuning
optimizer = optim.Adam(resnet18.parameters(), lr=1e-4)  # antes 1e-3
criterion = nn.CrossEntropyLoss()

In [ ]:
# =====================================
# FUNCIONES DE ENTRENAMIENTO Y EVALUACIÓN
# =====================================
def evaluate_model(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

def train_model(model, train_loader, test_loader, optimizer, criterion, epochs=10):
    history = {"train_loss": [], "test_acc": []}

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * inputs.size(0)

        avg_loss = total_loss / len(train_loader.dataset)
        acc = evaluate_model(model, test_loader)

        history["train_loss"].append(avg_loss)
        history["test_acc"].append(acc)

        print(f"Época {epoch}/{epochs} - Loss: {avg_loss:.4f} | Test Acc: {acc:.4f}")

    return history



In [ ]:
# =====================================
# ENTRENAMIENTO
# =====================================
print("\nEntrenando modelo ResNet18 (fine-tuning completo)...")
history = train_model(resnet18, train_loader, test_loader, optimizer, criterion, epochs=30)

final_acc = evaluate_model(resnet18, test_loader)
print(f"\n✅ Accuracy final de ResNet18 (fine-tuning): {final_acc:.2f}")

In [ ]:
# =====================================
# PLOT DE LOSS Y ACCURACY
# =====================================
epochs = range(1, len(history["train_loss"]) + 1)
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.plot(epochs, history["train_loss"], 'b-o')
plt.title("Pérdida (Loss) de entrenamiento")
plt.xlabel("Épocas")
plt.ylabel("Loss")

plt.subplot(1,2,2)
plt.plot(epochs, history["test_acc"], 'r-o')
plt.title("Exactitud (Accuracy) en Test")
plt.xlabel("Épocas")
plt.ylabel("Accuracy")

plt.tight_layout()
plt.show()